# Get the Data

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
import warnings
warnings.filterwarnings("ignore")

DATASET2_ODBC = 'PROTO.Melih.Dataset2_SimilarProducts'
DATASET2_TITLE = 'dataset2'

In [3]:
from common.sql_helper import custom_query

query = f'select * from {DATASET2_ODBC}'
result = custom_query(query=query)

result.head()

,WebSecenek,MarkaKodu,CinsiyetKodu,UrunGrubuKodu,Renk,cUrl,UrlNo
0,50189274-VR046,08,1,016,VR046,https://resim.aydinli.com.tr/08/1/016/S5018927...,2
1,50189277-VR033,08,1,016,VR033,https://resim.aydinli.com.tr/08/1/016/S5018927...,1
2,50189277-VR033,08,1,016,VR033,https://resim.aydinli.com.tr/08/1/016/S5018927...,2
3,50189278-VR085,08,1,016,VR085,https://resim.aydinli.com.tr/08/1/016/S5018927...,1
4,50189278-VR085,08,1,016,VR085,https://resim.aydinli.com.tr/08/1/016/S5018927...,2


In [4]:
result.shape

(125246, 7)

In [5]:
class_names = result[['CinsiyetKodu', 'UrunGrubuKodu']].drop_duplicates()
class_names.head()

,CinsiyetKodu,UrunGrubuKodu
0,1,016
63,2,016
83,2,057
93,2,013
111,1,013


In [6]:
class_names.shape

(50, 2)

In [ ]:
%%time
from common.model_helper import make_dataset_cv

errors = make_dataset_cv('08', 'DATASETDENEME', result,class_names , 1000)
errors

# Creating Model

## Creating Model CheckPoint

In [11]:
checkpoint_path = "checkpoints/TL_training_classifier/checkpoint.ckpt" # note: remember saving directly to Colab is temporary
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=True,
                                                         verbose=1)

## Flow Data

In [14]:
dir_ds_2 =  '../datasets/dataset2/brand_08/'

In [15]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_directory(dir_ds_2,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical',
                                              subset="training") # training dataset
test_generator = datagen.flow_from_directory(dir_ds_2,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode="categorical",
                                             subset="validation"
                                             ) # validation dataset

Found 850 images belonging to 50 classes.
Found 200 images belonging to 50 classes.


## Model 0: Building a transfer learning model using the Keras Functional API

In [13]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling2D")(x)
outputs = tf.keras.layers.Dense(len(class_names), name="output_layer")(x)

model_0 = tf.keras.Model(inputs, outputs)

In [16]:
model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy']
                )

history_0 = model_0.fit(train_generator,
                        epochs=5,
                        steps_per_epoch=len(train_generator),
                        validation_data=test_generator,
                        validation_steps=int(0.25 * len(test_generator)))

Epoch 1/5
27/27 [==============================] - 53s 2s/step - loss: 13.1926 - accuracy: 0.0082 - val_loss: 14.4904 - val_accuracy: 0.0000e+00
Epoch 2/5
27/27 [==============================] - 39s 1s/step - loss: 11.7251 - accuracy: 0.1765 - val_loss: 13.3862 - val_accuracy: 0.4688
Epoch 3/5
27/27 [==============================] - 39s 1s/step - loss: 14.3274 - accuracy: 0.6600 - val_loss: 16.1181 - val_accuracy: 0.6562
Epoch 4/5
27/27 [==============================] - 41s 2s/step - loss: 15.6020 - accuracy: 0.8659 - val_loss: 16.1181 - val_accuracy: 0.7812
Epoch 5/5
27/27 [==============================] - 38s 1s/step - loss: 15.5684 - accuracy: 0.8353 - val_loss: 9.2479 - val_accuracy: 0.4375


## Evaluation